In [1]:
import sklearn as sk
import pandas as pd
#Info for SKlearn algos 
#https://stackabuse.com/classification-in-python-with-scikit-learn-and-pandas/
#Info for XGBoost
#https://www.analyticsvidhya.com/blog/2021/07/15-most-important-features-of-scikit-learn/#:~:text=Scikit-Learn%20provides%20the%20functionality%20to%20convert%20text%20and,will%20explore%20in%20your%20journey%20of%20data%20science.
#LabelEncoder
#https://stackoverflow.com/questions/34915813/convert-text-columns-into-numbers-in-sklearn

In [2]:
filename = "diabetes_binary_5050split_health_indicators_BRFSS2015.csv"

In [3]:
data = pd.read_csv(filename) 


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

le = LabelEncoder()
data = data[data.columns[:]].apply(le.fit_transform)
#shuffling dataset since it is 50-50 even
data = shuffle(data)
data.head()


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
56039,1,0,1,1,20,0,0,0,1,0,...,1,1,3,30,15,0,0,6,5,7
57885,1,0,1,1,37,1,0,0,0,0,...,1,0,3,5,2,1,0,4,4,6
67346,1,0,0,1,27,1,0,0,1,1,...,1,0,4,30,10,0,1,11,3,4
5335,0,1,1,1,19,1,0,0,0,0,...,0,0,2,0,0,0,0,8,2,2
34240,0,0,0,1,8,0,0,0,1,0,...,1,0,1,0,0,0,0,2,5,7


In [5]:
data_train = data[:int(len(data)*0.80)]

data_test = data[int(len(data)*0.80):]

In [6]:
#training dataset
x_train =  data_train.iloc[:,1:]
y_train = data_train.iloc[:, 0]

#Test dataset
x_test =  data_test.iloc[:,1:]
y_test = data_test.iloc[:, 0]

In [7]:
y_train.head()

56039    1
57885    1
67346    1
5335     0
34240    0
Name: Diabetes_binary, dtype: int64

In [8]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=150, max_depth=10, random_state=0)
RF.fit(x_train, y_train)
#RF.predict(.iloc[460:,:])
round(RF.score(x_test,y_test), 4)

0.7485

In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import mean_squared_error
#xgb = XGBClassifier(colsample_bytree = 0.03, learning_rate = 0.01,max_depth = 20, alpha = 8, n_estimators = 500)
xgb = XGBClassifier()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.01,0.03,0.05], #so called `eta` value
              'max_depth': [6,7,8,9,11,12,13],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.3,0.5,0.7],
              'n_estimators': [5,50,100,300], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}


clf = GridSearchCV(xgb, parameters, n_jobs=10, 
                    cv=5,
                   scoring='roc_auc',
                   verbose=2, refit=True)
clf.fit(x_train,y_train)
#y_pred=xgb.predict(x_test)
round(clf.score(x_test,y_test), 4)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


C:\Users\Albert\anaconda3\envs\Machine learning projects\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:34:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[00:34:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.829

In [12]:
#best_parameters, score, _ = max(clf.cv_results_, key=lambda x: x[1])
#print('Raw AUC score:', score)
#for param_name in sorted(best_parameters.keys()):
    #print("%s: %r" % (param_name, best_parameters[param_name]))

In [13]:
round(clf.score(x_train,y_train), 4)

0.8459